In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from importlib import reload
import cv2
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

import albumentations
from albumentations import augmentations
import albumentations.pytorch

In [2]:
def feat_pool(feat: torch.tensor, operation: str):
    """
    Fuses the image's patches feature representation
    :param feat: The network object
    :param operation: Either max or mean for the pooling operation
    :returns: The final 256-D feature representation of the entire image
    """
    if operation == "max":
        return feat.max(axis=0)
    elif operation == "mean":
        return feat.mean(axis=0)
    else:
        raise Exception("The operation can be either mean or max")

In [2]:
df = pd.read_csv('all_features.csv')

In [4]:
class FeatSet(Dataset):
    def __init__(self, dataframe, mode, val_fold, test_fold):
        super().__init__()

        self.dataframe = dataframe
        self.mode = mode
        self.val_fold = val_fold
        self.test_fold = test_fold

        if self.mode == "train":
            rows = self.dataframe[~self.dataframe["fold"].isin([self.val_fold, self.test_fold])]
        elif self.mode == "val":
            rows = self.dataframe[self.dataframe["fold"] == self.val_fold]
        else:
            rows = self.dataframe[self.dataframe["fold"] == self.test_fold]

        self.data = []

        for row in tqdm(rows.values):
            _, label, _, _, _, feature = row

            feature_array = torch.load(feature)
            self.data.append((feature_array, label))

        np.random.shuffle(self.data)

        print(
            "\n\nreal:{}, fakes:{}, mode = {}".format(
                len(rows[rows["label"] == 0]), len(rows[rows["label"] == 1]), self.mode
            )
        )

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):

        return {
            "feat" : self.data[index][0],
            "label" : self.data[index][1]
        }

In [5]:
train_dataset = FeatSet(df, "train", 0, 1)
test_loader = DataLoader(test_dataset, batch_size=config.valid_batch_size, shuffle=True, num_workers=16, pin_memory=True, drop_last=False)

100%|██████████| 18684/18684 [00:06<00:00, 2909.54it/s]

real:10511, fakes:8173, mode = train



In [5]:
from classifier_dataset import Classifier_Dataset
import classifier
reload(classifier)
from classifier import Classifier_Linear

model = Classifier_Linear().cuda()

train_dataset = Classifier_Dataset(
    dataframe=df,
    mode="train",
    val_fold=0,
    test_fold=1
)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=8, pin_memory=True)

for batch in train_loader:
    tensors = batch["tensor"].cuda()
    target_labels = batch["label"].cuda()

    print(tensors.shape)
    print(target_labels.shape)

    output = model(tensors)
    print(output.shape)

    break

100%|██████████| 18684/18684 [00:05<00:00, 3148.58it/s]

real:10511, fakes:8173, mode = train

torch.Size([16, 5376])
torch.Size([16])
torch.Size([16, 1])
